# Explore data augmentations

This notebook will be used to test different data augmentation strategies. In your config file, you can modify the augmentations by adding `data_augmentation_options`. For example, I would like to add a random RGB to gray transform with a probability of 0.2:
```
data_augmentation_options {
    random_rgb_to_gray {
    probability: 0.2
    }
```

A complete list of augmentation is provided the [`preprocessor.proto` file of the Tf Object Detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto).

### Helper functions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

In [ ]:
def recenter_image(image):
    # ssd preprocessing
    image += [123.68, 116.779, 103.939]
    return image


def display_instances(image, bboxes, classes):
    # Cyclists show up as 0 in the one-hot encoding
    label_map = {1: "vehicle", 2: "pedestrian", 3: "Signs", 0: "Cyclists"}
    color_map = {1: "xkcd:fresh green", 2: "xkcd:cherry red", 3: "xkcd:azure", 0: "xkcd:butter yellow"}

    image = recenter_image(image)
    w, h, _ = image.shape
    # resize the bboxes
    bboxes[:, [0, 2]] *= w
    bboxes[:, [1, 3]] *= h

    f, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(image.astype(np.uint8))
    for bb, cl in zip(bboxes, classes):
        cl = int("".join([str(int(i)) for i in cl[::-1]]), 2)  # convert one hot class to int
        y1, x1, y2, x2 = bb
        rec = Rectangle((x1, y1), x2 - x1, y2 - y1, facecolor="none", edgecolor=color_map[cl], linewidth=2)
        # show the correct labels on the bounding boxes
        ax.text(x1, y1, label_map[cl], color=color_map[cl], size="large")
        ax.add_patch(rec)
    plt.show()


def display_batch(batch):
    # get images, bboxes and classes
    batched_images = batch[0]["image"].numpy()
    batched_bboxes = batch[1]["groundtruth_boxes"].numpy()
    batched_classes = batch[1]["groundtruth_classes"].numpy()
    num_bboxes = batch[1]["num_groundtruth_boxes"].numpy()
    batch_size = batched_images.shape[0]
    for idx in range(batch_size):
        display_instances(
            batched_images[idx, ...], batched_bboxes[idx, : num_bboxes[idx], :], batched_classes[idx, ...]
        )


### Display augmentations

In [ ]:
from utils import get_train_input

In [ ]:
train_dataset = get_train_input("training/experiment10/pipeline_new.config")

In [ ]:
%matplotlib inline

for batch in train_dataset.take(1):
    display_batch(batch)